In [ ]:
"""
1. Load faces from museum data set
2. Load base_model that will be transfered
    2.1. Freeze its layers
3. Build new model on top of it
4. Train new model with museum data set
5. TODO: Fine tune?
"""

In [6]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import fairnessMetrics as fm

In [7]:
# 
def readImages(folderName, image_size, batch_size):
    train_ds = keras.preprocessing.image_dataset_from_directory(
        folderName,
        validation_split=0.2,
        subset="training",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size
    )
    val_ds = keras.preprocessing.image_dataset_from_directory(
        folderName,
        validation_split=0.2,
        subset="validation",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size
    )
    return (train_ds, val_ds)

In [ ]:
# Load pretrained base model.
base_model = tf.keras.models.load_model('tmp/model') 

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(150, 150, 3))

# x = data_augmentation(inputs)  # TODO: Apply random data augmentation

In [ ]:
# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)

# What layers do we want to add?
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout

outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

In [ ]:
metrics_list = ["accuracy",
                tf.keras.metrics.TruePositives(), 
                tf.keras.metrics.TrueNegatives(),
                tf.keras.metrics.FalsePositives(), 
                tf.keras.metrics.FalseNegatives(), 
                fm.TruePositiveRate(),
                fm.TrueNegativeRate(),
                fm.FalsePositiveRate(),
                fm.FalseNegativeRate(),
                fm.PositivePredictedValue(),
                fm.FalseDiscoveryRate(),
                fm.NegativePredictedValue(),
                fm.FalseOmissionRate(),
                fm.BinaryDemographicParityDiff(),
                fm.DemographicParity(),
                fm.BinaryEqualizedOddsDiff(),
                fm.BinaryProportionalParityDiff(),
                fm.ProportionalParity(),
                fm.BinaryPredictiveRateParityDiff(),
                fm.PredictiveRateParity(),
                fm.BinaryAccuracyParityDiff(),
                fm.AccuracyParity(),
                fm.BinaryFalseNegativeRateParityDiff(),
                fm.BinaryFalsePositiveRateParityDiff(),
                fm.BinaryNegativePredictiveRateParityDiff(),
                fm.NegativePredictiveRateParity(),
                fm.BinarySpecificityParityDiff()]

In [ ]:
# Train new model
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=metrics_list,
)

epochs = 20
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)